<a href="https://colab.research.google.com/github/GarlandZhang/hairy_gan/blob/master/hairy_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"itsgarland2","key":"8ff807dec1f05dec1b97d62a072adef8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d jessicali9530/celeba-dataset --force
!unzip celeba-dataset.zip
!mv img_align_celeba celeba-dataset

In [4]:
!mv list_eval_partition.csv celeba-dataset/list_eval_partition.csv
!mv list_landmarks_align_celeba.csv celeba-dataset/list_landmarks_align_celeba.csv
!mv list_attr_celeba.csv celeba-dataset/list_attr_celeba.csv
!mv list_bbox_celeba.csv celeba-dataset/list_bbox_celeba.csv

In [5]:
import pandas as pd
import os
import shutil

In [6]:
complete_df = pd.read_csv('celeba-dataset/list_attr_celeba.csv')

In [7]:
partitions_df = pd.read_csv('celeba-dataset/list_eval_partition.csv') # 0 => train, 1 => validation, 2 => test
training_ids_df = partitions_df.loc[partitions_df['partition'] == 0]['image_id']

In [8]:
complete_df[complete_df['image_id'].isin(training_ids_df.tolist())]

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162765,162766.jpg,-1,-1,1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1
162766,162767.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1,1,1,-1,-1,-1,-1,1,1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,-1
162767,162768.jpg,1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,-1,-1,1,-1,-1,1,1,1,-1,-1,-1,-1,-1,1,1
162768,162769.jpg,-1,-1,-1,-1,-1,-1,1,1,1,-1,-1,-1,1,1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,1,-1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,1


In [9]:
black_hair_ids = complete_df.loc[complete_df['Black_Hair'] == 1]['image_id'].tolist() # filters out black hair candidates
blond_hair_ids = complete_df.loc[complete_df['Blond_Hair'] == 1]['image_id'].tolist() # filters out blond hair candidates

In [10]:
!mkdir celeba-dataset/trainA
!mkdir celeba-dataset/trainB

In [11]:
for id in black_hair_ids:
  shutil.copy(os.path.join('celeba-dataset/img_align_celeba', id), os.path.join('celeba-dataset', 'trainA', id))

for id in blond_hair_ids:
  shutil.copy(os.path.join('celeba-dataset/img_align_celeba', id), os.path.join('celeba-dataset', 'trainB', id))  

In [ ]:
!git clone https://www.github.com/keras-team/keras-contrib.git \
    && cd keras-contrib \
    && pip install git+https://www.github.com/keras-team/keras-contrib.git \
    && python convert_to_tf_keras.py \
    && USE_TF_KERAS=1 python setup.py install

In [18]:
from __future__ import print_function, division
import scipy
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam

import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import glob as glob

In [20]:
class DataLoader():
    def __init__(self, dataset_name, img_res):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('%s/%sB/*' % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [16]:
class HairyGan(): # based on CycleGan
  def __init__(self):
    self.img_rows = 128
    self.img_cols = 128
    self.img_channels = 3

    self.img_shape = (self.img_rows, self.img_cols, self.img_channels)
    
    patch = int(self.img_rows / 2**4)
    disc_out = (patch, patch, 1) # output shape of discriminator
    self.d_out_shape = (batch_size, ) + disc_out

    self.data_loader = DataLoader(dataset_name='celeba-dataset', img_res=(self.img_rows, self.img_cols))

    self.optimizer = Adam(0.0002, 0.5)

    # discriminators
    self.d_A = self.build_discriminator()
    self.d_B = self.build_discriminator()

    self.d_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
    self.d_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

    # generators
    self.g_AB = self.build_generator()
    self.g_BA = self.build_generator()

    self.combined_model = self.build_combined()

  def build_combined(self):
    lambda_cycle = 10.
    lambda_id = 0.9 * lambda_cycle

    img_A = Input(shape=self.img_shape) # image from domain A
    img_B = Input(shape=self.img_shape) # image from domain B

    fake_A = self.g_BA(img_B)
    fake_B = self.g_AB(img_A)
    
    reconstr_A = self.g_BA(fake_B)
    reconstr_B = self.g_AB(fake_A)

    id_A = self.g_BA(img_A)
    id_B = self.g_AB(img_B)

    valid_A = self.d_A(fake_A)
    valid_B = self.d_B(fake_B)

    combined = Model(
        inputs=[img_A, img_B],
        outputs=[valid_A, valid_B, reconstr_A, reconstr_B, id_A, id_B]
    )

    combined.compile(
        loss=['mse', 'mse', 'mae', 'mae', 'mae'],
        loss_weights=[1, 1, lambda_cycle, lambda_cycle, lambda_id, lambda_id],
        optimizer=self.optimizer
    )

    return combined

  def build_discriminator(self):

    def d_conv_block(layer_input, num_filt, f_size=4, normalization=True):
      d = Conv2D(num_filt, kernel_size=f_size, strides=2, padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
      if normalization:
        d = InstanceNormalization()(d)

      return d

    num_filt = 64

    img = Input(shape=self.img_shape)
    d1 = d_conv_block(img, num_filt, normalization=False)
    d2 = d_conv_block(d1, num_filt * 2)
    d3 = d_conv_block(d2, num_filt * 4)
    d4 = d_conv_block(d3, num_filt * 8)
    output = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, output)

  def build_generator(self):
    def g_conv_block(layer_input, num_filt, f_size=4):
      d = Conv2D(num_filt, kernel_size=f_size, strides=2, padding='same')(layer_input)
      d = LeakyReLU(alpha=0.2)(d)
      d = InstanceNormalization()(d)
      return d

    def g_deconv_block(layer_input, skip_input, num_filt, f_size=4, dropout_rate=0):
      u = UpSampling2D(size=2)(layer_input)
      u = Conv2D(num_filt, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
      
      if dropout_rate:
        u = Dropout(dropout_rate)(u)
      
      u = InstanceNormalization()(u)
      u = Concatenate()([u, skip_input])

      return u

    num_filt =  32

    d0 = Input(shape=self.img_shape)
    d1 = g_conv_block(d0, num_filt)
    d2 = g_conv_block(d1, num_filt * 2)
    d3 = g_conv_block(d2, num_filt * 4)
    d4 = g_conv_block(d3, num_filt * 8)

    u1 = g_deconv_block(d4, d3, num_filt * 4)
    u2 = g_deconv_block(u1, d2, num_filt * 2)
    u3 = g_deconv_block(u2, d1, num_filt)

    u4 = UpSampling2D(size=2)(u3)
    output = Conv2D(self.img_channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model([d0, output])


  def train(self, epochs, batch_size=1, sample_interval=50):
    valid = np.ones(self.d_out_shape)
    fake = np.zeros(self.d_out_shape)

    for epoch in range(epochs):
      for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
        fake_A = self.g_BA(imgs_B)
        fake_B = self.g_AB(imgs_A)

        real_A_loss = self.d_A.train_on_batch(imgs_A, valid)
        real_B_loss = self.d_B.train_on_batch(imgs_B, valid)

        fake_A_loss = self.d_A.train_on_batch(fake_A, fake)
        fake_B_loss = self.d_B.train_on_batch(fake_B, fake)

        d_A_loss = 0.5 * np.add(real_A_loss, fake_A_loss)
        d_B_loss = 0.5 * np.add(real_B_loss, fake_B_loss)

        d_loss = 0.5 * np.add(d_A_loss, d_B_loss)

        g_loss = self.combined_model.train_on_batch(
            [imgs_A, imgs_B],
            [fake, fake, imgs_A, imgs_B, imgs_A, imgs_B]
        )

        if batch_i % sample_interval == 0:
          self.sample_images(epoch, batch_i)

  def sample_images(self, epoch, batch_i):
    pass

    